In [9]:
%load_ext autoreload
%autoreload 2
from notebook import *
# if get something about NUMEXPR_MAX_THREADS being set incorrectly, don't worry.  It's not a problem.

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Myth of programming

Consider the following code snippets, which side is faster?

In [2]:
compare([do_render_code("swap.c", show="regswap"),do_render_code("swap.c", show="xorswap")])

In [3]:
render_code("swap.c")

// swap.c:1-89 (89 lines)
#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>

void (*swap)(int* a, int* b);
void regswap(int* a, int* b)
{
    int temp = *a;
    *a = *b;
    *b = temp;
}

void xorswap(int* a, int* b)
{
    *a ^= *b;
    *b ^= *a;
    *a ^= *b;
}

/* This function takes last element as pivot, places
   the pivot element at its correct position in sorted
    array, and places all smaller (smaller than pivot)
   to left of pivot and all greater elements to right
   of pivot */
int partition (int arr[], int low, int high)
{
    int pivot = arr[high];    // pivot
    int i = (low - 1);  // Index of smaller element
 
    for (int j = low; j <= high- 1; j++)
    {
        // If current element is smaller than or
        // equal to pivot
        if (arr[j] <= pivot)
        {
            i++;    // increment index of smaller element
            if(i!=j)
            swap(&arr[i], &arr[j]);
        }
    }
            if(i+1 != high)
    swap(&arr[i + 1], &arr[high]);
    return (i + 1);
}
 
/* The main function that implements QuickSort
 arr[] --> Array to be sorted,
  low  --> Starting index,
  high  --> Ending index */
void quickSort(int arr[], int low, int high)
{
    if (low < high)
    {
        /* pi is partitioning index, arr[p] is now
           at right place */
        int pi = partition(arr, low, high);
 
        // Separately sort elements before
        // partition and after partition
        quickSort(arr, low, pi - 1);
        quickSort(arr, pi + 1, high);
    }
}
 

int main(int argc, char **argv)
{
    unsigned array_size = 131072;
    int *data;
    struct timeval time_start, time_end;
    // command: swap array_size method
    if(argc > 1)
        array_size = atoi(argv[1]);
    data = (int *)malloc(sizeof(int)*array_size);
    if(argc > 2 && atoi(argv[2])==1)
        swap = xorswap;
    else
        swap = regswap;
    for (unsigned i = 0; i < array_size; ++i)
        data[i] = rand();
   gettimeofday(&time_start, NULL);
   quickSort(data, 0, array_size-1);
/*   for (unsigned i = 0; i < array_size; ++i)
        swap(&data[i],&data[array_size-1-i]);*/
   gettimeofday(&time_end, NULL);
   fprintf(stderr, "data[rand()] = %d\n", data[rand()%array_size]);
   fprintf(stderr, "sorted %lf seconds\n",((time_end.tv_sec * 1000000 + time_end.tv_usec) - (time_start.tv_sec * 1000000 + time_start.tv_usec))/1000000.0);
   return 0;
}

In [5]:
! make clean; make swap
! echo "Version A"
! ./swap 262144 0; ./swap 262144 0;  ./swap 262144 0;  ./swap 262144 0;  ./swap 262144 0; 
! echo "Version B"
#./arithmetics_2_B; ./arithmetics_2_B; ./arithmetics_2_B; ./arithmetics_2_B; ./arithmetics_2_B
!./swap 262144 1;./swap 262144 1;./swap 262144 1;./swap 262144 1;./swap 262144 1;

rm -f arithmetics arithmetics_A arithmetics_B flip_array swap
cc -O3 -DHAVE_LINUX_PERF_EVENT_H swap.c -o swap
Version A
data[rand()] = 1969641983
sorted 0.017979 seconds
data[rand()] = 1969641983
sorted 0.018622 seconds
data[rand()] = 1969641983
sorted 0.019304 seconds
data[rand()] = 1969641983
sorted 0.022966 seconds
data[rand()] = 1969641983
sorted 0.020625 seconds
Version B
data[rand()] = 1969641983
sorted 0.021799 seconds
data[rand()] = 1969641983
sorted 0.020581 seconds
data[rand()] = 1969641983
sorted 0.021440 seconds
data[rand()] = 1969641983
sorted 0.022624 seconds
data[rand()] = 1969641983
sorted 0.023230 seconds


In [7]:
! gcc -S -O3 swap.c

In [8]:
render_code("swap.s")

// swap.s:1-363 (363 lines)
	.file	"swap.c"
	.text
	.p2align 4
	.globl	regswap
	.type	regswap, @function
regswap:
.LFB39:
	.cfi_startproc
	endbr64
	movl	(%rdi), %eax
	movl	(%rsi), %edx
	movl	%edx, (%rdi)
	movl	%eax, (%rsi)
	ret
	.cfi_endproc
.LFE39:
	.size	regswap, .-regswap
	.p2align 4
	.globl	xorswap
	.type	xorswap, @function
xorswap:
.LFB40:
	.cfi_startproc
	endbr64
	movl	(%rdi), %eax
	xorl	(%rsi), %eax
	movl	%eax, (%rdi)
	xorl	(%rsi), %eax
	movl	%eax, (%rsi)
	xorl	%eax, (%rdi)
	ret
	.cfi_endproc
.LFE40:
	.size	xorswap, .-xorswap
	.p2align 4
	.globl	partition
	.type	partition, @function
partition:
.LFB41:
	.cfi_startproc
	endbr64
	pushq	%r15
	.cfi_def_cfa_offset 16
	.cfi_offset 15, -16
	movslq	%edx, %rax
	leal	-1(%rsi), %ecx
	pushq	%r14
	.cfi_def_cfa_offset 24
	.cfi_offset 14, -24
	movq	%rdi, %r14
	pushq	%r13
	.cfi_def_cfa_offset 32
	.cfi_offset 13, -32
	pushq	%r12
	.cfi_def_cfa_offset 40
	.cfi_offset 12, -40
	pushq	%rbp
	.cfi_def_cfa_offset 48
	.cfi_offset 6, -48
	pushq	%rbx
	.cfi_def_cfa_offset 56
	.cfi_offset 3, -56
	movq	%rax, %rbx
	subq	$24, %rsp
	.cfi_def_cfa_offset 80
	movl	%eax, 4(%rsp)
	leaq	(%rdi,%rax,4), %rax
	movq	%rax, 8(%rsp)
	movl	(%rax), %ebp
	cmpl	%esi, %ebx
	jle	.L9
	movslq	%esi, %rdx
	notl	%esi
	leal	(%rsi,%rbx), %r13d
	leaq	1(%rdx), %r15
	addq	%r15, %r13
	jmp	.L7
	.p2align 4,,10
	.p2align 3
.L13:
	movslq	%r12d, %rdx
	leaq	(%r14,%rdx,4), %rdi
	call	*swap(%rip)
	movl	%r12d, %ecx
	movl	%ebx, %r12d
.L6:
	movq	%r15, %rdx
	cmpq	%r15, %r13
	je	.L5
.L14:
	addq	$1, %r15
.L7:
	leaq	(%r14,%rdx,4), %rsi
	leal	1(%rcx), %r12d
	cmpl	%ebp, (%r14,%rdx,4)
	jg	.L6
	leal	2(%rcx), %ebx
	cmpl	%edx, %r12d
	jne	.L13
	movl	%r12d, %ecx
	movq	%r15, %rdx
	movl	%ebx, %r12d
	cmpq	%r15, %r13
	jne	.L14
	.p2align 4,,10
	.p2align 3
.L5:
	cmpl	%r12d, 4(%rsp)
	je	.L4
	movslq	%ecx, %rcx
	movq	8(%rsp), %rsi
	leaq	4(%r14,%rcx,4), %rdi
	call	*swap(%rip)
.L4:
	addq	$24, %rsp
	.cfi_remember_state
	.cfi_def_cfa_offset 56
	movl	%r12d, %eax
	popq	%rbx
	.cfi_def_cfa_offset 48
	popq	%rbp
	.cfi_def_cfa_offset 40
	popq	%r12
	.cfi_def_cfa_offset 32
	popq	%r13
	.cfi_def_cfa_offset 24
	popq	%r14
	.cfi_def_cfa_offset 16
	popq	%r15
	.cfi_def_cfa_offset 8
	ret
	.p2align 4,,10
	.p2align 3
.L9:
	.cfi_restore_state
	movl	%esi, %r12d
	jmp	.L5
	.cfi_endproc
.LFE41:
	.size	partition, .-partition
	.p2align 4
	.globl	quickSort
	.type	quickSort, @function
quickSort:
.LFB42:
	.cfi_startproc
	endbr64
	cmpl	%edx, %esi
	jge	.L20
	pushq	%r13
	.cfi_def_cfa_offset 16
	.cfi_offset 13, -16
	movq	%rdi, %r13
	pushq	%r12
	.cfi_def_cfa_offset 24
	.cfi_offset 12, -24
	movl	%edx, %r12d
	pushq	%rbp
	.cfi_def_cfa_offset 32
	.cfi_offset 6, -32
	movl	%esi, %ebp
	pushq	%rbx
	.cfi_def_cfa_offset 40
	.cfi_offset 3, -40
	subq	$8, %rsp
	.cfi_def_cfa_offset 48
.L17:
	movl	%ebp, %esi
	movl	%r12d, %edx
	movq	%r13, %rdi
	call	partition
	movl	%ebp, %esi
	movq	%r13, %rdi
	movl	%eax, %ebx
	leal	-1(%rax), %edx
	call	quickSort
	leal	1(%rbx), %ebp
	cmpl	%r12d, %ebp
	jl	.L17
	addq	$8, %rsp
	.cfi_def_cfa_offset 40
	popq	%rbx
	.cfi_def_cfa_offset 32
	popq	%rbp
	.cfi_def_cfa_offset 24
	popq	%r12
	.cfi_def_cfa_offset 16
	popq	%r13
	.cfi_def_cfa_offset 8
	ret
	.p2align 4,,10
	.p2align 3
.L20:
	.cfi_restore 3
	.cfi_restore 6
	.cfi_restore 12
	.cfi_restore 13
	ret
	.cfi_endproc
.LFE42:
	.size	quickSort, .-quickSort
	.section	.rodata.str1.1,"aMS",@progbits,1
.LC0:
	.string	"data[rand()] = %d\n"
.LC2:
	.string	"sorted %lf seconds\n"
	.section	.text.startup,"ax",@progbits
	.p2align 4
	.globl	main
	.type	main, @function
main:
.LFB43:
	.cfi_startproc
	endbr64
	pushq	%r14
	.cfi_def_cfa_offset 16
	.cfi_offset 14, -16
	pushq	%r13
	.cfi_def_cfa_offset 24
	.cfi_offset 13, -24
	pushq	%r12
	.cfi_def_cfa_offset 32
	.cfi_offset 12, -32
	pushq	%rbp
	.cfi_def_cfa_offset 40
	.cfi_offset 6, -40
	pushq	%rbx
	.cfi_def_cfa_offset 48
	.cfi_offset 3, -48
	subq	$48, %rsp
	.cfi_def_cfa_offset 96
	movq	%fs:40, %rax
	movq	%rax, 40(%rsp)
	xorl	%eax, %eax
	cmpl	$1, %edi
	jg	.L38
	movl	$524288, %edi
	movl	$131072, %r12d
	call	malloc@PLT
